<a href="https://colab.research.google.com/github/sggyuan/GNN/blob/main/%E6%9C%80%E7%BB%88%E6%95%B0%E6%8D%AE%E7%89%88%E6%9C%ACpyg%E8%BE%93%E5%85%A5vgae%E7%A4%BA%E4%BE%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.4.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 38.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
data = torch.load('/content/drive/MyDrive/my_graph_data-4.pt')

<ipython-input-5-737c7aca6486>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('/content/drive/MyDrive/my_graph_data-4.pt')


In [6]:
data.train_mask = data.val_mask = data.test_mask = None
data

Data(x=[11241, 20], edge_index=[2, 80252], y=[11241])

In [7]:
data = train_test_split_edges(data)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [8]:
data

Data(x=[11241, 20], y=[11241], val_pos_edge_index=[2, 2006], test_pos_edge_index=[2, 4012], train_pos_edge_index=[2, 68214], train_neg_adj_mask=[11241, 11241], val_neg_edge_index=[2, 2006], test_neg_edge_index=[2, 4012])

In [9]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(GCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv2 = GCNConv(2 * out_channels, out_channels, cached=True) # cached only for transductive learning

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)


In [10]:
from torch_geometric.nn import GAE

In [16]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
# parameters
out_channels = 2
num_features = data.num_features
epochs = 100

# model
model = GAE(GCNEncoder(num_features, out_channels))

# move to GPU (if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)
train_pos_edge_index = data.train_pos_edge_index.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, 'max', patience=10, factor=0.5)

In [13]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    #if args.variational:
    #   loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [17]:
for epoch in range(1, epochs + 1):
    loss = train()

    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))

Epoch: 001, AUC: 0.5000, AP: 0.5000
Epoch: 002, AUC: 0.5000, AP: 0.5000
Epoch: 003, AUC: 0.5000, AP: 0.5000
Epoch: 004, AUC: 0.5000, AP: 0.5000
Epoch: 005, AUC: 0.5000, AP: 0.5000
Epoch: 006, AUC: 0.5000, AP: 0.5000
Epoch: 007, AUC: 0.5000, AP: 0.5000
Epoch: 008, AUC: 0.5000, AP: 0.5000
Epoch: 009, AUC: 0.5000, AP: 0.5000
Epoch: 010, AUC: 0.5000, AP: 0.5000
Epoch: 011, AUC: 0.5000, AP: 0.5000
Epoch: 012, AUC: 0.5000, AP: 0.5000
Epoch: 013, AUC: 0.5001, AP: 0.5001
Epoch: 014, AUC: 0.5002, AP: 0.5001
Epoch: 015, AUC: 0.5002, AP: 0.5001
Epoch: 016, AUC: 0.5004, AP: 0.5002
Epoch: 017, AUC: 0.5004, AP: 0.5002
Epoch: 018, AUC: 0.5004, AP: 0.5002
Epoch: 019, AUC: 0.5004, AP: 0.5002
Epoch: 020, AUC: 0.5004, AP: 0.5002
Epoch: 021, AUC: 0.5004, AP: 0.5002
Epoch: 022, AUC: 0.5004, AP: 0.5002
Epoch: 023, AUC: 0.5004, AP: 0.5002
Epoch: 024, AUC: 0.5004, AP: 0.5002
Epoch: 025, AUC: 0.5004, AP: 0.5002
Epoch: 026, AUC: 0.5004, AP: 0.5002
Epoch: 027, AUC: 0.5004, AP: 0.5002
Epoch: 028, AUC: 0.5004, AP: